In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

# gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
# tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
# BI : Função de mapeamento do Henrick nas extremidades
# BI2: Função identidade nas extremidades

nomes = [
    "WENO-JS" , "WENO-Z" , "WENO-Z+" , "WENO-D", "WENO-A"
]
cores = [
    "#A41127",            # WENO-JS         : Vermelho
    "#273EAF", "#8EA9F1", # WENO-Z e WENO-Z+: Azul
    "#41631D", "#9CD87B"  # WENO-D e WENO-A : Verde
]

API      = API_Numpy
equation = euler_equation
γ        = 1.4

WENOs = {}

WENOs[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 2]] = simulation(API, equation, WENO_Zp_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 3]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 4]] = simulation(API, equation, WENO_A_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim

Weights = {}

Weights[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 2]] = simulation(API, equation, WENO_Zp_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 3]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 4]] = simulation(API, equation, WENO_A_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights


In [3]:

def cond_inicial(problema):
    
    if problema == "1-Lax":
        
        # Riemann problem of Lax
        #-------------------------------------------------------------------
        t_final   = 0.13
        n         = 200
        interval  = [-0.5, 0.5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------

    if problema == "2-Sod":

        # Riemann problem of Sod
        #-------------------------------------------------------------------
        t_final   = 2
        n         = 200
        interval  = [-5, 5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------

    if problema == "3-Shock Density Wave Titarev-Toro":
        
        # Shock density wave Titarev–Toro
        #-------------------------------------------------------------------
        t_final   = 5
        n         = 1500
        interval  = [-10, 5]
        fator     = 8
        n_ref     = n*fator
        fronteira = TitarevToroBoundary
        #-------------------------------------------------------------------
        
    if problema == "4-Shock Density Wave Shu-Osher":
        
        # Shock density wave Shu–Osher
        #-------------------------------------------------------------------
        t_final   = 1.8
        n         = 200
        interval  = [-5, 5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------
        
    if problema == "5-Blast Wave":
        
        # Blast wave
        #-------------------------------------------------------------------
        t_final   = 0.038
        n         = 400
        interval  = [0, 1]
        fator     = 8
        n_ref     = n*fator
        fronteira = FronteiraReflexiva
        #-------------------------------------------------------------------
        
    Δx     = (interval[1]-interval[0])/n
    Δx_ref = (interval[1]-interval[0])/n_ref

    x     = tf.range(interval[0], interval[1], Δx    , dtype=dtype)
    x_ref = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    CFL = 0.5
    γ   = 1.4

    if problema == "1-Lax":
        
        # Riemann problem of Lax
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = np.where(x < 0, 0.445, 0.5)
            u = np.where(x < 0, 0.698, 0.0)
            p = np.where(x < 0, 3.528, 0.5710)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q

        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "2-Sod":
        
        # Riemann problem of Sod
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = np.where(x < 0, 0.125, 1)
            u = x-x
            p = np.where(x < 0, 0.1  , 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "3-Shock Density Wave Titarev-Toro":
    
        # Shock density wave Titarev–Toro
        #-------------------------------------------------------------------
        def f_test(x):
            k = 20
            ρ = np.where(x < -4.5, 1.515695, 1+np.sin(k*np.pi*x)/10)
            u = np.where(x < -4.5, 0.523346, 0)
            p = np.where(x < -4.5, 1.805000, 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "4-Shock Density Wave Shu-Osher":
                
        # Shock density wave Shu–Osher
        #-------------------------------------------------------------------
        def f_test(x):
            k = 5
            ρ = np.where(x < -4, 3.857143, 1+0.2*np.sin(k*x))
            u = np.where(x < -4, 2.629369, 0)
            p = np.where(x < -4, 31/3    , 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "5-Blast Wave":
        
        # Blast wave
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = x-x+1
            u = x-x
            p = np.where(  x < 0.1, 1000, 0.01)
            p = np.where(0.9 <   x,  100,    p)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------
    
    init = {
        "Q"        : Q         ,
        "Q_ref"    : Q_ref     ,
        "t_final"  : t_final   ,
        "Δx"       : Δx        ,
        "Δx_ref"   : Δx_ref    ,
        "CFL"      : CFL       ,
        "fronteira": fronteira ,
        "interval" : interval  ,
        "fator"    : fator
    }
    
    return init

In [4]:
problemas = [
    "1-Lax"                            ,
    "2-Sod"                            ,
    "3-Shock Density Wave Titarev-Toro",
    "4-Shock Density Wave Shu-Osher"   ,
    "5-Blast Wave"
]

u     = {}
pesos = {}

for problema in problemas:
    
    init = cond_inicial(problema)
    
    Q         = init["Q"]
    t_final   = init["t_final"]
    Δx        = init["Δx"]
    CFL       = init["CFL"]
    fronteira = init["fronteira"]
    
    for nome in nomes:
        
        u[problema + ": " + nome]     = Q
        pesos[problema + ": " + nome] = []
        pesos[problema + ": " + nome].append(Weights[nome](u[problema + ": " + nome], Δx, fronteira)[4][0,...])
        
        for t in np.arange(10):
            u[problema + ": " + nome] = WENOs[nome](u[problema + ": " + nome], t_final/10, Δx, CFL, fronteira)
            pesos[problema + ": " + nome].append(Weights[nome](u[problema + ": " + nome], Δx, fronteira)[4][0,...])
        
        pesos[problema + ": " + nome]  = np.concatenate(pesos[problema + ": " + nome], axis=0)
        pesos[problema + ": " + nome] /= np.sum(pesos[problema + ": " + nome], axis=-1, keepdims = True)
        
        print(problema[2:] + ": " + nome + "               ")


Lax: WENO-JS               
Lax: WENO-Z               
Lax: WENO-Z+               
Lax: WENO-D               
Lax: WENO-A               
Sod: WENO-JS               
Sod: WENO-Z               
Sod: WENO-Z+               
Sod: WENO-D               
Sod: WENO-A               
Shock Density Wave Titarev-Toro: WENO-JS               
Shock Density Wave Titarev-Toro: WENO-Z               
Shock Density Wave Titarev-Toro: WENO-Z+               
Shock Density Wave Titarev-Toro: WENO-D               
Shock Density Wave Titarev-Toro: WENO-A               
Shock Density Wave Shu-Osher: WENO-JS               
Shock Density Wave Shu-Osher: WENO-Z               
Shock Density Wave Shu-Osher: WENO-Z+               
Shock Density Wave Shu-Osher: WENO-D               
Shock Density Wave Shu-Osher: WENO-A               
Blast Wave: WENO-JS               
Blast Wave: WENO-Z               
Blast Wave: WENO-Z+               
Blast Wave: WENO-D               
Blast Wave: WENO-A               


In [6]:
cores = [
    "#fcffa4", "#f6d746", "#fca50a", "#f37819", "#dd513a",
    "#bc3754", "#932667", "#6a176e", "#420a68", "#160b39",
    "#000004"
]

for problema in problemas:
    for nome in nomes:
        
        x     = pesos[problema + ": " + nome][...,0]
        y     = pesos[problema + ": " + nome][...,2]
        label = np.repeat([
            "t=0", "t=1", "t=2", "t=3", "t=4", 
            "t=5", "t=6", "t=7", "t=8", "t=9", 
            "t=10"], x.shape[0]/11)

        df = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y     ,
                "label" : label
            }
        )
        fig = ply_exp.scatter(
            data_frame = df    ,
            x          = x     ,
            y          = y     ,
            color      = label ,
            color_discrete_sequence = cores
        )
        fig.add_trace(
            ply_go.Scatter(
                x    = np.concatenate([
                    tf.range(0, 1, 0.01, dtype=dtype)  ,
                    tf.range(0, 1, 0.01, dtype=dtype)*0,
                    tf.range(0, 1, 0.01, dtype=dtype)
                ]),
                y    = np.concatenate([
                      tf.range(0, 1, 0.01, dtype=dtype)*0,
                      tf.range(0, 1, 0.01, dtype=dtype)  ,
                    1-tf.range(0, 1, 0.01, dtype=dtype)
                ]),
                mode = "lines",
                line = ply_go.scatter.Line(color="#666666"),
                name = ""
            )
        )
        fig.update_yaxes(exponentformat="power")
        # fig.show()
        fig.write_html("htmls/Mapa dos Pesos " + problema[2:] + " " + nome + ".html")

In [11]:
# BI : Função de mapeamento do Henrick nas extremidades
# BI2: Função identidade nas extremidades

nomes = [
    "WENO-JS" , "WENO-Z" , "WENO-Z+" , "WENO-D", "WENO-A"
]
cores = [
    "#A41127",            # WENO-JS         : Vermelho
    "#273EAF", "#8EA9F1", # WENO-Z e WENO-Z+: Azul
    "#41631D", "#9CD87B"  # WENO-D e WENO-A : Verde
]

API      = API_Numpy
equation = transp_equation
γ        = 1.4

WENOs = {}

WENOs[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 2]] = simulation(API, equation, WENO_Zp_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 3]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 4]] = simulation(API, equation, WENO_A_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim

Weights = {}

Weights[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 2]] = simulation(API, equation, WENO_Zp_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 3]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 4]] = simulation(API, equation, WENO_A_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights


In [12]:
Δx = 0.01

def f_test(x):
    z = tf.constant(-0.7, dtype=dtype)
    δ = tf.constant(0.005, dtype=dtype)
    β = tf.math.log(tf.constant(2.0, dtype=dtype))/(36.0*(δ**2.0))
    a = tf.constant(0.5, dtype=dtype)
    α = tf.constant(10, dtype=dtype)
    
    def G(x, β, z):
        return tf.math.exp(-β*(x-z)**2)
    
    def F(x, α, a):
        return tf.math.sqrt(tf.math.maximum(1-(α**2)*((x-a)**2),0))
    
    f1 = (G(x, β, z-δ) + 4*G(x, β, z) + G(x, β, z+δ))/6 # x in [-0.8, -0.6]
    f2 = 1                                              # x in [-0.4, -0.2]
    f3 = 1-tf.math.abs(10*(x-0.1))                      # x in [ 0.0,  0.2]
    f4 = (F(x, α, a-δ) + 4*F(x, α, a) + F(x, α, a+δ))/6 # x in [ 0.4,  0.6]
    f5 = 0                                              # otherwise
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.8),
        tf.math.less_equal(x, -0.6)
    )
    f1 = f1*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.4),
        tf.math.less_equal(x, -0.2)
    )
    f2 = f2*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.0),
        tf.math.less_equal(x, 0.2)
    )
    f3 = f3*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.4),
        tf.math.less_equal(x, 0.6)
    )
    f4 = f4*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    f = f1 + f2 + f3 + f4 + f5
    
    return f

In [13]:
t_final = 2
n       = 400
CFL     = 0.2

fronteira = FronteiraPeriodica

Δx     = 2/n
x      = tf.range(-1, 1, Δx, dtype=dtype)

u     = {}
pesos = {}

for nome in nomes:

    u["Formas: " + nome]     = f_test(x)
    pesos["Formas: " + nome] = []
    pesos["Formas: " + nome].append(Weights[nome](u["Formas: " + nome], Δx, fronteira)[4])

    for t in np.arange(10):
        u["Formas: " + nome] = WENOs[nome](u["Formas: " + nome], t_final/10, Δx, CFL, fronteira)
        pesos["Formas: " + nome].append(Weights[nome](u["Formas: " + nome], Δx, fronteira)[4])

    pesos["Formas: " + nome]  = np.concatenate(pesos["Formas: " + nome], axis=0)
    pesos["Formas: " + nome] /= np.sum(pesos["Formas: " + nome], axis=-1, keepdims = True)

    print("Formas: " + nome + "               ")

Formas: WENO-JS               
Formas: WENO-Z               
Formas: WENO-Z+               
Formas: WENO-D               
Formas: WENO-A               


In [14]:
cores = [
    "#fcffa4", "#f6d746", "#fca50a", "#f37819", "#dd513a",
    "#bc3754", "#932667", "#6a176e", "#420a68", "#160b39",
    "#000004"
]

for nome in nomes:

    x     = pesos["Formas: " + nome][...,0]
    y     = pesos["Formas: " + nome][...,2]
    label = np.repeat([
        "t=0", "t=1", "t=2", "t=3", "t=4", 
        "t=5", "t=6", "t=7", "t=8", "t=9", 
        "t=10"], x.shape[0]/11)

    df = pd.DataFrame(
        {
            "x"     : x     ,
            "y"     : y     ,
            "label" : label
        }
    )
    fig = ply_exp.scatter(
        data_frame = df    ,
        x          = x     ,
        y          = y     ,
        color      = label ,
        color_discrete_sequence = cores
    )
    fig.add_trace(
        ply_go.Scatter(
            x    = np.concatenate([
                tf.range(0, 1, 0.01, dtype=dtype)  ,
                tf.range(0, 1, 0.01, dtype=dtype)*0,
                tf.range(0, 1, 0.01, dtype=dtype)
            ]),
            y    = np.concatenate([
                  tf.range(0, 1, 0.01, dtype=dtype)*0,
                  tf.range(0, 1, 0.01, dtype=dtype)  ,
                1-tf.range(0, 1, 0.01, dtype=dtype)
            ]),
            mode = "lines",
            line = ply_go.scatter.Line(color="#666666"),
            name = ""
        )
    )
    fig.update_yaxes(exponentformat="power")
    # fig.show()
    fig.write_html("htmls/Mapa dos Pesos Formas " + nome + ".html")